In [0]:
jdbcHostname = "server4traffic.database.windows.net"
jdbcDatabase = "datawarehouse04"
jdbcPort = 1433
jdbcUsername = "admin2traffic"
jdbcPassword = "abcd1234@"

In [0]:
# URL JDBC
jdbcUrl = f"jdbc:sqlserver://{jdbcHostname}:{jdbcPort};database={jdbcDatabase};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

# Paramètres de connexion
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import TimestampType


In [0]:
dbutils.widgets.text("date_param", "")
date_a_traiter = dbutils.widgets.get("date_param")

In [0]:
# construire la requête SQL avec filtre
query = f"(SELECT * FROM raw.transactions WHERE CAST(timestamp_insertion AS DATE) = '{date_a_traiter}') AS transactions_filtrés"

# lire uniquement les données de la veille
df_transactions = spark.read.jdbc(
    url=jdbcUrl,
    table=query,
    properties=connectionProperties
)

In [0]:
# # Lire une table SQL dans un DataFrame Spark
# df_transactions = spark.read.jdbc(url=jdbcUrl, table="raw.transactions", properties=connectionProperties)

In [0]:
display(df_transactions.limit(10))

In [0]:
# Compter les lignes où AU MOINS UNE colonne est null
nb_lignes_null = df_transactions.filter(
    F.col("store_id").isNull() |
    F.col("date").isNull() |
    F.col("heure").isNull() |
    F.col("transactions").isNull() |
    F.col("chiffre_affaires").isNull()
).count()

In [0]:
print(f"Nombre de lignes avec au moins un null : {nb_lignes_null}")

In [0]:
df_transactions = df_transactions.dropna()


In [0]:
clean_transactions = df_transactions.withColumn(
    "date", F.coalesce(
        F.to_date("date", "yyyy-MM-dd"),
        F.to_date("date", "yyyy/MM/dd"),
        F.to_date("date", "dd/MM/yyyy")
    )
).withColumn(
    "store_id", F.col("store_id").cast("int")
).withColumn(
    "heure", F.hour(F.col("heure").cast(TimestampType()))
).withColumn(
    "nb_transactions", F.col("transactions").cast("int")
).withColumn(
    "chiffre_affaires", F.round(F.col("chiffre_affaires").cast("double"), 2)
).select(
    "store_id", "date", "heure", "nb_transactions", "chiffre_affaires"
)
display(clean_transactions)

In [0]:
clean_transactions.createOrReplaceTempView("clean_transactions")

In [0]:
spark.sql("""
    SELECT 
        *
    FROM clean_transactions
    where nb_transactions> 1000
    ORDER BY nb_transactions
""").show(10)

In [0]:
# Filtrage des données
clean_transactions = clean_transactions.filter(
    (F.col("store_id") <= 10) &
    (F.col("heure").between(9, 19)) &
    (F.col("date").isNotNull()) &
    (F.col("nb_transactions").between(1, 1000)) &
    (F.col("chiffre_affaires").between(1, 50000))
)
clean_transactions.createOrReplaceTempView("clean_transactions")

display(clean_transactions)

In [0]:
transactions_manquants = spark.sql("""
    -- table des dates attendues
    WITH heures_attendues AS (
        SELECT explode(sequence(9, 19)) AS heure
    ), 
    dates_attendues AS (
        select d.date from
        (SELECT explode(sequence(DATE '2023-04-27', DATE '2025-04-25', 
        INTERVAL 1 DAY)
        ) AS date) as d
        where dayofweek(d.date) BETWEEN 2 AND 6
    ),
    -- ids des magasins et des capteurs
    identifiants_uniques AS (
        SELECT DISTINCT store_id
        FROM clean_transactions
    ),
    toutes_combinaisons AS (
        SELECT i.store_id, d.date, h.heure
        FROM identifiants_uniques i
        CROSS JOIN dates_attendues d
        CROSS JOIN heures_attendues h
    )

    select t.store_id, t.date,t.heure
    from toutes_combinaisons t
    left join clean_transactions v
    on v.store_id = t.store_id
    and v.date = t.date
    and v.heure = t.heure
    where v.heure is null

""")
transactions_manquants.createOrReplaceTempView("transactions_manquants")
display(transactions_manquants)



In [0]:
transactions_manquants = spark.sql("""
    with moyenne_transactions as (
    select v.store_id, 
    v.date, 
    cast(avg(v.nb_transactions) AS int) as nb_transactions,
    cast(avg(v.chiffre_affaires) AS decimal(10,2)) as chiffre_affaires
    from clean_transactions v
    group by v.store_id, v.date
    )
    
    select v.*, mt.nb_transactions, mt.chiffre_affaires
    from transactions_manquants v
    inner join moyenne_transactions mt
    on v.store_id = mt.store_id
    and v.date = mt.date
""")
transactions_manquants.createOrReplaceTempView("transactions_manquants")
display(transactions_manquants)

In [0]:
clean_transactions = spark.sql("""
    SELECT store_id, date, heure, nb_transactions, chiffre_affaires
    FROM clean_transactions

    UNION ALL

    SELECT store_id, date, heure, nb_transactions, chiffre_affaires
    FROM transactions_manquants
""")
display(transactions_manquants)


In [0]:
clean_transactions.groupBy("date", "heure", "store_id") \
    .agg(F.count("*").alias("nb")) \
    .filter(F.col("nb") > 1) \
    .show()

In [0]:
clean_transactions.count()

In [0]:
clean_transactions = clean_transactions.dropDuplicates(
    ["date", "heure", "store_id"]
)

In [0]:
clean_transactions.count()

In [0]:
clean_transactions.select(F.min(F.col("date")), F.max(F.col("date"))).show()

In [0]:
try:
    clean_transactions.write.jdbc(
        url=jdbcUrl,
        table="analytics.transactions",
        mode="append",  # ou "overwrite" selon usage
        properties=connectionProperties
    )
    print("Données insérées avec succès dans analytics.transactions.")
except Exception as e:
    print("Erreur lors de l'insertion dans la base SQL :")
    print(str(e))